In [1]:
import numpy as np
np.set_printoptions(precision=3)
np.set_printoptions(suppress=True)
np.set_printoptions(linewidth=60)

import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import style
style.use('ggplot')
import itertools

from stanhelper import stanhelper

%matplotlib inline
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import sys

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import numpy.random as npr
from sklearn.cluster import KMeans
from sklearn import linear_model
from scipy.stats import invgamma
from scipy import sparse, stats

plt.style.use('ggplot')

In [2]:
import random
import time
randseed = 60382071
print("random seed: ", randseed)
random.seed(randseed)
np.random.seed(randseed)

random seed:  60382071


In [3]:
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn import linear_model

In [4]:
OUT_DATA_DIR = '../res/factorfit/'
DATA_DIR = '../dat/tmdb_proc/'

# load data

In [5]:
tr_vd_data = pd.read_csv(os.path.join(DATA_DIR, 'train_full.csv'))
data = tr_vd_data['rating']
row = tr_vd_data['uid']
col = tr_vd_data['sid']
n_users, n_items = (2828, 901)
X = sparse.coo_matrix((data, (row, col)), shape=(n_users, n_items)).toarray()

In [6]:
unique_uid = np.loadtxt(os.path.join(DATA_DIR, 'unique_uid.txt'))
user2id = dict((uid, i) for (i, uid) in enumerate(unique_uid))
tmdb = pd.read_csv(os.path.join(DATA_DIR, "tmdb_clean.csv"))
tmdbrev = tmdb[['revenue','movie_id', 'budget', \
                'original_language', 'new_movie_id']]
tmdbrev['uid'] = [user2id[k] for k in tmdbrev['movie_id']] # map movie_id to u_id in the substitute confounder
tmdbrev = tmdbrev.sort_values(['uid'])

/Users/yixinwang/py2/lib/python2.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [7]:
revenue = tmdbrev['revenue']
y = np.log(revenue)

In [8]:
pcaU = np.loadtxt(OUT_DATA_DIR + '/cause_pca_k50_U.csv')
pmfU = np.loadtxt(OUT_DATA_DIR + '/cause_pmf_k50_U.csv')
defU = np.loadtxt(OUT_DATA_DIR + '/cause_def_k_xpost.csv')

In [9]:
cov = tmdb[['budget', 'runtime']]

In [10]:
y_scaler = preprocessing.StandardScaler().fit(y[:,np.newaxis])
y_scaled = y_scaler.fit_transform(y[:,np.newaxis]).T[0]

In [11]:
X_scaler = preprocessing.StandardScaler().fit(X)
X_scaled = X_scaler.fit_transform(X)

/Users/yixinwang/py2/lib/python2.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [12]:
pcaU_scaler = preprocessing.StandardScaler().fit(pcaU)
pcaU_scaled = pcaU_scaler.fit_transform(pcaU)

In [13]:
pmfU_scaler = preprocessing.StandardScaler().fit(pmfU)
pmfU_scaled = pmfU_scaler.fit_transform(pmfU)

In [14]:
defU_scaler = preprocessing.StandardScaler().fit(defU)
defU_scaled = defU_scaler.fit_transform(defU)

In [15]:
cov_scaler = preprocessing.StandardScaler().fit(cov)
cov_scaled = cov_scaler.fit_transform(cov)

In [16]:
X_train, X_test, \
cov_train, cov_test, \
pcaU_train, pcaU_test, \
pmfU_train, pmfU_test, \
defU_train, defU_test, \
y_train, y_test, tmdbrev_train, tmdbrev_test = \
    train_test_split(X_scaled, cov_scaled, pcaU_scaled, pmfU_scaled, defU_scaled, \
                     y_scaled, tmdbrev, test_size=0.20, random_state=randseed)

# estimate causal effect

## no control

In [17]:
nctrl_sing = np.zeros(n_items)
reg = linear_model.Ridge(alpha=0.0)
for i in range(n_items):
    reg.fit(X_train[:,i][:,np.newaxis], y_train)
    nctrl_sing[i] = reg.coef_

## control for covariates

In [18]:
covctrl_sing = np.zeros(n_items)
reg = linear_model.Ridge(alpha=0.0)
for i in range(n_items):
    reg.fit(np.column_stack([X_train[:,i], cov_train]), y_train)
    covctrl_sing[i] = reg.coef_[0]

## control for PCA substitute confounder

In [19]:
pcactrl_sing = np.zeros(n_items)
reg = linear_model.Ridge(alpha=0.0)
for i in range(n_items):
    reg.fit(np.column_stack([X_train[:,i], pcaU_train]), y_train)
    pcactrl_sing[i] = reg.coef_[0]

## control for PMF substitute confounder

In [20]:
pmfctrl_sing = np.zeros(n_items)
reg = linear_model.Ridge(alpha=0.0)
for i in range(n_items):
    reg.fit(np.column_stack([X_train[:,i], pmfU_train]), y_train)
    pmfctrl_sing[i] = reg.coef_[0]

## control for DEF substitute confounder

In [21]:
defctrl_sing = np.zeros(n_items)
reg = linear_model.Ridge(alpha=0.0)
for i in range(n_items):
    reg.fit(np.column_stack([X_train[:,i] - defU_train[:,i]]), y_train)
    defctrl_sing[i] = reg.coef_[0]

## control for PCA substitute confounder and covariates

In [22]:
pcacovctrl_sing = np.zeros(n_items)
reg = linear_model.Ridge(alpha=0.0)
for i in range(n_items):
    reg.fit(np.column_stack([X_train[:,i], pcaU_train, cov_train]), y_train)
    pcacovctrl_sing[i] = reg.coef_[0]

## control for PMF substitute confounder and covariates

In [23]:
pmfcovctrl_sing = np.zeros(n_items)
reg = linear_model.Ridge(alpha=0.0)
for i in range(n_items):
    reg.fit(np.column_stack([X_train[:,i], pmfU_train, cov_train]), y_train)
    pmfcovctrl_sing[i] = reg.coef_[0]

## control for DEF substitute confounder and covariates

In [24]:
defcovctrl_sing = np.zeros(n_items)
reg = linear_model.Ridge(alpha=0.0)
for i in range(n_items):
    reg.fit(np.column_stack([X_train[:,i] - defU_train[:,i], cov_train]), y_train)
    defcovctrl_sing[i] = reg.coef_[0]

# evaluate log likelihood

In [25]:
def logpdf(y, ypred, sigma=1.3):
    return -0.5 * np.log(2*np.pi) - 0.5 * (y-ypred)**2 / sigma**2

In [26]:
nctrllpdfs = logpdf(y_test, np.matmul(np.column_stack([X_test]), nctrl_sing))
covctrllpdfs = logpdf(y_test, np.matmul(np.column_stack([X_test]), covctrl_sing))
pcactrllpdfs = logpdf(y_test, np.matmul(np.column_stack([X_test]), pcactrl_sing))
pmfctrllpdfs = logpdf(y_test, np.matmul(np.column_stack([X_test]), pmfctrl_sing))
defctrllpdfs = logpdf(y_test, np.matmul(np.column_stack([X_test]), defctrl_sing))
pcacovctrllpdfs = logpdf(y_test, np.matmul(np.column_stack([X_test]), pcacovctrl_sing))
pmfcovctrllpdfs = logpdf(y_test, np.matmul(np.column_stack([X_test]), pmfcovctrl_sing))
defcovctrllpdfs = logpdf(y_test, np.matmul(np.column_stack([X_test]), defcovctrl_sing))

## regular test set

In [27]:
print("no control", nctrllpdfs.mean())
print("control for X", covctrllpdfs.mean())
print("control for PCA confounders", pcactrllpdfs.mean())
print("control for PMF confounders", pmfctrllpdfs.mean())
print("control for DEF confounders", defctrllpdfs.mean())
print("control for PCA confounders and X", pcacovctrllpdfs.mean())
print("control for PMF confounders and X", pmfcovctrllpdfs.mean())
print("control for DEF confounders and X", defcovctrllpdfs.mean())

no control -1.4942115171598733
control for X -1.5108222305439594
control for PCA confounders -1.2592246089467007
control for PMF confounders -1.3087380809092464
control for DEF confounders -1.229431034258638
control for PCA confounders and X -1.2601227861835826
control for PMF confounders and X -1.3167451645857071
control for DEF confounders and X -1.2372676433173477


## non-English movies

In [28]:
spoken_df = pd.read_csv(os.path.join(DATA_DIR, "spoken_languages_clean.csv"))
spoken_df = spoken_df[["new_movie_id", "iso_639_1"]].sort_values("new_movie_id")
spoken_dummy = pd.get_dummies(spoken_df["iso_639_1"])
spoken_dummy["new_movie_id"] = spoken_df[["new_movie_id"]]
spoken_dummy = spoken_dummy.groupby("new_movie_id").sum().reset_index()
testset_slang = tmdbrev_test.merge(spoken_dummy, on="new_movie_id")
nonenslang = (testset_slang['en']==0)

In [29]:
print("no control", nctrllpdfs[nonenslang].mean())
print("control for X", covctrllpdfs[nonenslang].mean())
print("control for PCA confounders", pcactrllpdfs[nonenslang].mean())
print("control for PMF confounders", pmfctrllpdfs[nonenslang].mean())
print("control for DEF confounders", defctrllpdfs[nonenslang].mean())
print("control for PCA confounders and X", pcacovctrllpdfs[nonenslang].mean())
print("control for PMF confounders and X", pmfcovctrllpdfs[nonenslang].mean())
print("control for DEF confounders and X", defcovctrllpdfs[nonenslang].mean())

no control -2.222961562984115
control for X -2.254504173940256
control for PCA confounders -1.0710504969888004
control for PMF confounders -1.1242292842486086
control for DEF confounders -0.9962851210880637
control for PCA confounders and X -1.0610292404779338
control for PMF confounders and X -1.115203922008078
control for DEF confounders and X -0.9821726425934653


## non-drama/comedy/action movies

In [30]:
genres_df = pd.read_csv(os.path.join(DATA_DIR, "genres_df_clean.csv"))
genres_df = genres_df[["new_movie_id", "genre"]].sort_values("new_movie_id")
genres_dummy = pd.get_dummies(genres_df["genre"])
genres_dummy["new_movie_id"] = genres_df[["new_movie_id"]]
genres_dummy = genres_dummy.groupby("new_movie_id").sum().reset_index()
testset_genres = tmdbrev_test.merge(genres_dummy, on="new_movie_id")
testset_genres["unpopular_genres"] = \
    (1-testset_genres["Drama"])*(1-testset_genres["Comedy"])*(1-testset_genres["Action"])
unpop = (testset_genres["unpopular_genres"]==1)

In [31]:
print("no control", nctrllpdfs[unpop].mean())
print("control for X", covctrllpdfs[unpop].mean())
print("control for PCA confounders", pcactrllpdfs[unpop].mean())
print("control for PMF confounders", pmfctrllpdfs[unpop].mean())
print("control for DEF confounders", defctrllpdfs[unpop].mean())
print("control for PCA confounders and X", pcacovctrllpdfs[unpop].mean())
print("control for PMF confounders and X", pmfcovctrllpdfs[unpop].mean())
print("control for DEF confounders and X", defcovctrllpdfs[unpop].mean())

no control -1.825719594297616
control for X -1.7568856798104904
control for PCA confounders -1.2855852016943037
control for PMF confounders -1.345983098440941
control for DEF confounders -1.2973511769877217
control for PCA confounders and X -1.3125887368141143
control for PMF confounders and X -1.3546141406017849
control for DEF confounders and X -1.299284688229018
